In [1]:
from IPython import get_ipython
from IPython.core.display import display
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
get_ipython().run_line_magic('run', 'setup')

bevolking = CBS.bevolking().iloc[0]
#display(bevolking.BevolkingOpDeEersteVanDeMaand)
#display(bevolking['per 100k'])

def addstats(df):
  global dagoverzicht

  # de aanname is dat df al gegroepeerd is op datum, en dat de kolommen de dagtotalen zijn
  dagoverzicht = dagoverzicht.merge(df, how='left', left_index=True, right_index=True)
  for stat in df.columns:
    # nieuw = dagtotaal, zet missende dagtotalen op 0
    dagoverzicht[f'{stat} (nieuw)'] = dagoverzicht[stat].fillna(0).astype(int)
    # kolom met eerder de dagtotalen = cumulatieve som over dagtotalen
    dagoverzicht[stat] = dagoverzicht[f'{stat} (nieuw)'].cumsum()
    # factor voor 100k
    dagoverzicht[f'{stat} per 100.000'] = dagoverzicht[stat] * bevolking['per 100k']
    # verschil met 7 dagen terug
    dagoverzicht[f'{stat} 7d'] = dagoverzicht[f'{stat} (nieuw)'].rolling(7).sum().fillna(0)
    # en weer factor 100 k
    dagoverzicht[f'{stat} 7d per 100.000'] = dagoverzicht[f'{stat} 7d'] * bevolking['per 100k']

/tmp/ipykernel_1760/2095093918.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


/home/runner/work/corona-locator-nederland/corona-locator-nederland/setup.py:146: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.DataFrame(r['value']))


In [2]:
@run
def cell():
  df = ArcGIS.nice('f27f743476a142538e8054f7a7ce12e1')

  df['date'] = pd.to_datetime(df.date.str.replace(' .*', '', regex=True))
  df.set_index('date', inplace=True)

  # base date range uit NICE
  global dagoverzicht
  # maak leeg dataframe met een rij voor elke dag van 2020-02-27 tm Date_of_publication
  dagoverzicht = pd.DataFrame(index=pd.date_range(start='2020-02-27', end=df.index.max()))
  # noem de index Key
  dagoverzicht.index.name='Key'
  # vul de datum kolom
  dagoverzicht['Datum'] = dagoverzicht.index.strftime('%Y-%m-%d')
  # vaste waarde voor LandCode
  dagoverzicht['LandCode'] = 'NL'

  # the NICE data as prepared by ESRI can just be used as-is, so just rename the columns
  for prefix, kind in [ ('ic', 'IC'), ('zkh', 'Ziekenhuis') ]:
    df = df.rename(columns={
      f'{prefix}NewIntake': f'NICE {kind} Bedden (intake)',
      f'{prefix}IntakeCount': f'NICE {kind} Bedden',
      f'{prefix}IntakeCumulative': f'NICE {kind} Bedden (cumulatief)',
      f'{prefix}DiedCumulative': f'NICE {kind} Overleden',
    })

  # remove the columns we don't use
  df = df[[col for col in df.columns if 'NICE' in col]]
  dagoverzicht = dagoverzicht.merge(df, how='left', left_index=True, right_index=True)

  # after the merge, dates missing in the ESRI/NICE set will be `nan`, so fill these
  for col in df.columns:
    # for cumulatief columns, fill-forward from last known value
    if 'cumulatief' in col:
      dagoverzicht[col] = dagoverzicht[col].ffill()
    # for non-cumulative and whatever remains in cumulative after fill forward (which will be leading nan's), fill with 0
    dagoverzicht[col] = dagoverzicht[col].fillna(0)

  for kind in [ 'IC', 'Ziekenhuis' ]:
    dagoverzicht[f'NICE {kind} Bedden (toename)'] = (dagoverzicht[f'NICE {kind} Bedden'] - dagoverzicht[f'NICE {kind} Bedden'].shift(1)).fillna(0)

  for window, shift, past in [(3, 1, 'afgelopen '), (7, 0, '')]:
    dagoverzicht[f'NICE IC Bedden (intake) {past}{window}d'] = dagoverzicht['NICE IC Bedden (intake)'].shift(shift).rolling(window).sum().fillna(0)

  dagoverzicht['NICE IC Bedden (intake) week-1'] = dagoverzicht['NICE IC Bedden (intake) 7d'].shift(7).fillna(0)

downloading nice/f27f743476a142538e8054f7a7ce12e1_0-2022-03-22@15-49.csv


nice f27f743476a142538e8054f7a7ce12e1_0 zipping nice/f27f743476a142538e8054f7a7ce12e1_0-2022-03-22@15-49.csv


removing obsolete nice/f27f743476a142538e8054f7a7ce12e1_0-2022-03-22@15-28.csv.gz


loading nice/f27f743476a142538e8054f7a7ce12e1_0-2022-03-22@15-49.csv.gz


In [3]:
@run
def cell():
  df = RIVM.csv('COVID-19_aantallen_gemeente_per_dag').rename(columns={
    'Total_reported': 'Positief getest',
    'Deceased': 'Overleden',
    'Date_of_publication': 'Datum',
    'Date_of_report': 'Today',
  })
  # sloop tijd van de datum en zet om in datetime object
  df['Datum'] = pd.to_datetime(df.Datum.str.replace(' .*', '', regex=True))

  # sommeer pos en overl op datum en voeg toe aan dagoverzicht
  addstats(df.groupby(['Datum']).agg({'Positief getest': 'sum', 'Overleden': 'sum'}))
  global dagoverzicht
  for col in ['Overleden', 'Positief getest']:
    dagoverzicht[f'{col} week-1'] = dagoverzicht[f'{col} 7d'].shift(7).fillna(0)
  display(dagoverzicht.head(10))

rivm/COVID-19_aantallen_gemeente_per_dag-2022-03-22@14-15.csv exists


loading rivm/COVID-19_aantallen_gemeente_per_dag-2022-03-22@14-15.csv.gz


,Datum,LandCode,NICE IC Bedden (intake),NICE IC Bedden (cumulatief),NICE IC Bedden,NICE IC Overleden,NICE Ziekenhuis Bedden (intake),NICE Ziekenhuis Bedden (cumulatief),NICE Ziekenhuis Bedden,NICE Ziekenhuis Overleden,...,Positief getest (nieuw),Positief getest per 100.000,Positief getest 7d,Positief getest 7d per 100.000,Overleden (nieuw),Overleden per 100.000,Overleden 7d,Overleden 7d per 100.000,Overleden week-1,Positief getest week-1
Key,,,,,,,,,,,,,,,,,,,,,
2020-02-27,2020-02-27,NL,1,13,13,0,2,64,61,0,...,0,0.000000,0.0,0.000000,0,0.000000,0.0,0.000000,0.0,0.0
2020-02-28,2020-02-28,NL,0,14,14,0,4,68,65,0,...,1,0.005683,0.0,0.000000,0,0.000000,0.0,0.000000,0.0,0.0
2020-02-29,2020-02-29,NL,1,15,15,0,1,69,65,0,...,1,0.011365,0.0,0.000000,0,0.000000,0.0,0.000000,0.0,0.0
2020-03-01,2020-03-01,NL,2,17,17,0,7,76,72,0,...,3,0.028413,0.0,0.000000,0,0.000000,0.0,0.000000,0.0,0.0
2020-03-02,2020-03-02,NL,0,17,16,0,10,87,83,0,...,5,0.056826,0.0,0.000000,0,0.000000,0.0,0.000000,0.0,0.0
2020-03-03,2020-03-03,NL,2,19,18,0,8,95,90,0,...,4,0.079556,0.0,0.000000,0,0.000000,0.0,0.000000,0.0,0.0
2020-03-04,2020-03-04,NL,3,22,21,0,10,108,102,0,...,10,0.136382,24.0,0.136382,0,0.000000,0.0,0.000000,0.0,0.0
2020-03-05,2020-03-05,NL,1,24,21,0,11,119,111,0,...,17,0.232986,41.0,0.232986,0,0.000000,0.0,0.000000,0.0,0.0
2020-03-06,2020-03-06,NL,4,28,24,0,21,138,129,0,...,40,0.460290,80.0,0.454607,0,0.000000,0.0,0.000000,0.0,0.0


In [4]:
@run
def cell():
  df = RIVM.csv('COVID-19_ziekenhuisopnames').rename(columns={
    'Hospital_admission': 'Ziekenhuisopnames',
    'Date_of_statistics': 'Datum',
  })
  # datum naar datetime
  df['Datum'] = pd.to_datetime(df.Datum.str.replace(' .*', '', regex=True))
  # sommeer op datum en voeg toe aan dagoverzicht
  addstats(df.groupby(['Datum']).agg({'Ziekenhuisopnames': 'sum'}))
  dagoverzicht['Ziekenhuisopnames week-1'] = dagoverzicht['Ziekenhuisopnames 7d'].shift(7).fillna(0)
  display(dagoverzicht.head())

rivm/COVID-19_ziekenhuisopnames-2022-03-22@14-16.csv exists


loading rivm/COVID-19_ziekenhuisopnames-2022-03-22@14-16.csv.gz


,Datum,LandCode,NICE IC Bedden (intake),NICE IC Bedden (cumulatief),NICE IC Bedden,NICE IC Overleden,NICE Ziekenhuis Bedden (intake),NICE Ziekenhuis Bedden (cumulatief),NICE Ziekenhuis Bedden,NICE Ziekenhuis Overleden,...,Overleden 7d,Overleden 7d per 100.000,Overleden week-1,Positief getest week-1,Ziekenhuisopnames,Ziekenhuisopnames (nieuw),Ziekenhuisopnames per 100.000,Ziekenhuisopnames 7d,Ziekenhuisopnames 7d per 100.000,Ziekenhuisopnames week-1
Key,,,,,,,,,,,,,,,,,,,,,
2020-02-27,2020-02-27,NL,1,13,13,0,2,64,61,0,...,0.0,0.0,0.0,0.0,3,3,0.017048,0.0,0.0,0.0
2020-02-28,2020-02-28,NL,0,14,14,0,4,68,65,0,...,0.0,0.0,0.0,0.0,7,4,0.039778,0.0,0.0,0.0
2020-02-29,2020-02-29,NL,1,15,15,0,1,69,65,0,...,0.0,0.0,0.0,0.0,9,2,0.051143,0.0,0.0,0.0
2020-03-01,2020-03-01,NL,2,17,17,0,7,76,72,0,...,0.0,0.0,0.0,0.0,17,8,0.096604,0.0,0.0,0.0
2020-03-02,2020-03-02,NL,0,17,16,0,10,87,83,0,...,0.0,0.0,0.0,0.0,27,10,0.153430,0.0,0.0,0.0


In [5]:
@run
def cell():
  global dagoverzicht

  datasets = [
    ('COVID-19_reproductiegetal', 'Rt_avg', 'Reproductiegetal'),
    ('COVID-19_prevalentie', 'prev_avg', 'Besmettelijk'),
  ]
  for dataset, source, target in datasets:
    # laad de dataset
    df = RIVM.json(dataset).rename(columns={source: target})
    # Date naar datetime index voor de merge
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    # merge alleen de target kolom
    df = df[[target]]
    # voeg to aan dagoverzicht
    dagoverzicht = dagoverzicht.merge(df, how='left', left_index=True, right_index=True)
    # vul lege waarden met 0
    dagoverzicht[target] = dagoverzicht[target].fillna(0)
  # per 100k factor
  dagoverzicht['Besmettelijk per 100.000'] = (dagoverzicht['Besmettelijk']  * bevolking['per 100k']).round(0)
  display(dagoverzicht)

rivm/COVID-19_reproductiegetal-2022-03-22@14-16.json exists


loading rivm/COVID-19_reproductiegetal-2022-03-22@14-16.json.gz


rivm/COVID-19_prevalentie-2022-03-22@14-16.json exists


loading rivm/COVID-19_prevalentie-2022-03-22@14-16.json.gz


,Datum,LandCode,NICE IC Bedden (intake),NICE IC Bedden (cumulatief),NICE IC Bedden,NICE IC Overleden,NICE Ziekenhuis Bedden (intake),NICE Ziekenhuis Bedden (cumulatief),NICE Ziekenhuis Bedden,NICE Ziekenhuis Overleden,...,Positief getest week-1,Ziekenhuisopnames,Ziekenhuisopnames (nieuw),Ziekenhuisopnames per 100.000,Ziekenhuisopnames 7d,Ziekenhuisopnames 7d per 100.000,Ziekenhuisopnames week-1,Reproductiegetal,Besmettelijk,Besmettelijk per 100.000
Key,,,,,,,,,,,,,,,,,,,,,
2020-02-27,2020-02-27,NL,1,13,13,0,2,64,61,0,...,0.0,3,3,0.017048,0.0,0.000000,0.0,2.06,6938.0,39.0
2020-02-28,2020-02-28,NL,0,14,14,0,4,68,65,0,...,0.0,7,4,0.039778,0.0,0.000000,0.0,2.10,8697.0,49.0
2020-02-29,2020-02-29,NL,1,15,15,0,1,69,65,0,...,0.0,9,2,0.051143,0.0,0.000000,0.0,1.99,10866.0,62.0
2020-03-01,2020-03-01,NL,2,17,17,0,7,76,72,0,...,0.0,17,8,0.096604,0.0,0.000000,0.0,1.83,13451.0,76.0
2020-03-02,2020-03-02,NL,0,17,16,0,10,87,83,0,...,0.0,27,10,0.153430,0.0,0.000000,0.0,1.79,16539.0,94.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-18,2022-03-18,NL,15,17774,161,4900,225,90267,2151,11379,...,484337.0,102644,215,583.283852,1766.0,10.035455,1649.0,0.00,0.0,0.0
2022-03-19,2022-03-19,NL,14,17789,163,4907,174,90437,2024,11397,...,477320.0,102802,158,584.181702,1717.0,9.757008,1709.0,0.00,0.0,0.0
2022-03-20,2022-03-20,NL,16,17805,162,4912,151,90586,2039,11404,...,463186.0,102936,134,584.943169,1616.0,9.183067,1742.0,0.00,0.0,0.0


In [6]:
@run
def cell():
  df = RIVM.csv('COVID-19_uitgevoerde_testen').rename(columns={
    'Date_of_statistics': 'Datum',
    'Tested_with_result': 'GGD getest',
    'Tested_positive': 'GGD getest positief',
  })
  df['Datum'] = pd.to_datetime(df.Datum)
  df = df.groupby(['Datum']).agg({'GGD getest': 'sum', 'GGD getest positief': 'sum'})
  display(df)

  global dagoverzicht
  columns = dagoverzicht.columns

  dagoverzicht = dagoverzicht.merge(df, how='left', left_index=True, right_index=True)

  dagoverzicht['GGD percentage positief'] = (dagoverzicht['GGD getest positief'] / dagoverzicht['GGD getest']).fillna(0)

  dagoverzicht['GGD getest (7 daags)'] = dagoverzicht['GGD getest'].rolling(7).sum().fillna(0)
  dagoverzicht['GGD getest positief (7 daags)'] = dagoverzicht['GGD getest positief'].rolling(7).sum().fillna(0)

  dagoverzicht['GGD percentage positief (7 daags)'] = (dagoverzicht['GGD getest positief (7 daags)'] / dagoverzicht['GGD getest (7 daags)']).fillna(0)

  dagoverzicht['GGD getest (cumulatief)'] = dagoverzicht['GGD getest'].cumsum()
  dagoverzicht['GGD getest positief (cumulatief)'] = dagoverzicht['GGD getest positief'].cumsum()

  dagoverzicht['GGD percentage positief (cumulatief)'] = (dagoverzicht['GGD getest positief (cumulatief)'] / dagoverzicht['GGD getest (cumulatief)']).fillna(0)

  # fill 0 *after* the calculations above to prevent summing 'GGD getest (7 daags) from the starting date of the 'GGD getest' series
  for col in dagoverzicht.columns:
    # only zero out new columns
    if col in columns: continue
    dagoverzicht[col] = dagoverzicht[col].fillna(0)

  display(dagoverzicht.head(10))

rivm/COVID-19_uitgevoerde_testen-2022-03-22@14-16.csv exists


loading rivm/COVID-19_uitgevoerde_testen-2022-03-22@14-16.csv.gz


,GGD getest,GGD getest positief
Datum,,
2020-06-01,1551,73
2020-06-02,6821,203
2020-06-03,8880,165
2020-06-04,9352,173
2020-06-05,9478,135
...,...,...
2022-03-16,73358,48032
2022-03-17,64138,42126
2022-03-18,52399,34913


,Datum,LandCode,NICE IC Bedden (intake),NICE IC Bedden (cumulatief),NICE IC Bedden,NICE IC Overleden,NICE Ziekenhuis Bedden (intake),NICE Ziekenhuis Bedden (cumulatief),NICE Ziekenhuis Bedden,NICE Ziekenhuis Overleden,...,Besmettelijk per 100.000,GGD getest,GGD getest positief,GGD percentage positief,GGD getest (7 daags),GGD getest positief (7 daags),GGD percentage positief (7 daags),GGD getest (cumulatief),GGD getest positief (cumulatief),GGD percentage positief (cumulatief)
Key,,,,,,,,,,,,,,,,,,,,,
2020-02-27,2020-02-27,NL,1,13,13,0,2,64,61,0,...,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-28,2020-02-28,NL,0,14,14,0,4,68,65,0,...,49.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-29,2020-02-29,NL,1,15,15,0,1,69,65,0,...,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-01,2020-03-01,NL,2,17,17,0,7,76,72,0,...,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-02,2020-03-02,NL,0,17,16,0,10,87,83,0,...,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-03,2020-03-03,NL,2,19,18,0,8,95,90,0,...,115.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-04,2020-03-04,NL,3,22,21,0,10,108,102,0,...,138.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-05,2020-03-05,NL,1,24,21,0,11,119,111,0,...,164.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-06,2020-03-06,NL,4,28,24,0,21,138,129,0,...,192.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
@run
def cell():
  # laad dataset
  df = LCPS.csv('covid-19-datafeed').rename(columns={  
    'IC_Bedden_COVID_Nederland': 'LCPS IC Bedden COVID',
    'IC_Bedden_COVID_Internationaal': 'LCPS IC Bedden COVID Internationaal',
    'IC_Bedden_Non_COVID_Nederland': 'LCPS IC Bedden Non COVID',
    'Kliniek_Bedden_Nederland': 'LCPS Kliniek Bedden COVID',
    'IC_Nieuwe_Opnames_COVID_Nederland': 'LCPS IC Nieuwe Opnames COVID',
    'Kliniek_Nieuwe_Opnames_COVID_Nederland': 'LCPS Kliniek Nieuwe Opnames COVID',
  })
  # datum naar datetime index voor merge
  df['Datum'] = pd.to_datetime(df['Datum'], format='%d-%m-%Y')
  df.set_index('Datum', inplace=True)

  # sommeer op datum
  df = df.groupby(['Datum']).agg({col: 'sum' for col in df.columns})

  # toename = waarde - vorige
  df['LCPS IC Bedden COVID (toename)'] = (df['LCPS IC Bedden COVID'] - df['LCPS IC Bedden COVID'].shift(1)).fillna(0)
  df['LCPS Kliniek Bedden COVID (toename)'] = (df['LCPS Kliniek Bedden COVID'] - df['LCPS Kliniek Bedden COVID'].shift(1)).fillna(0)

  global dagoverzicht
  # voeg toe aan dagoverzicht
  dagoverzicht = dagoverzicht.merge(df, how='left', left_index=True, right_index=True)
  for col in df.columns:
    # vervang lege waarden door 0
    dagoverzicht[col] = dagoverzicht[col].fillna(0).astype(int)

lcps/covid-19-datafeed-2022-03-22@13-00.csv exists


loading lcps/covid-19-datafeed-2022-03-22@13-00.csv.gz


In [8]:
@run
def cell():
  # laad corrections van mzelst
  df = GitHub.csv('mzelst/covid-19/contents/corrections/corrections_perday.csv')
  # date naar datetime index voor merge
  df['date'] = pd.to_datetime(df.date)
  df.set_index('date', inplace=True)
  columns =  {
    'net.infection': 'Positief getest (toename)',
    'net.hospitals': 'Ziekenhuisopnames (toename)',
    'net.deaths': 'Overleden (toename)'
  }
  # rename kolommen naar onze namen
  df = df.rename(columns=columns)[list(columns.values())]

  global dagoverzicht
  # voeg toe aan dagoverxicht
  dagoverzicht = dagoverzicht.merge(df, how='left', left_index=True, right_index=True)
  for col in columns.values():
    # set lege waarden op 0
    dagoverzicht[col] = dagoverzicht[col].fillna(0).astype(int)

https://api.github.com/repos/mzelst/covid-19/contents/corrections/corrections_perday.csv


downloading github/corrections_perday-2022-03-22@14-54.csv


github corrections_perday zipping github/corrections_perday-2022-03-22@14-54.csv


2 github corrections_perday files, pruning 1, keeping 1


removed github/corrections_perday-2022-03-20@14-59.csv.gz


In [9]:
@run
def cell():
  global dagoverzicht
  dagoverzicht['Personen'] = bevolking.BevolkingOpDeEersteVanDeMaand
  dagoverzicht['Key'] = dagoverzicht.index.strftime('%Y-%m-%d')
  display(dagoverzicht)

,Datum,LandCode,NICE IC Bedden (intake),NICE IC Bedden (cumulatief),NICE IC Bedden,NICE IC Overleden,NICE Ziekenhuis Bedden (intake),NICE Ziekenhuis Bedden (cumulatief),NICE Ziekenhuis Bedden,NICE Ziekenhuis Overleden,...,LCPS Kliniek Bedden COVID,LCPS IC Nieuwe Opnames COVID,LCPS Kliniek Nieuwe Opnames COVID,LCPS IC Bedden COVID (toename),LCPS Kliniek Bedden COVID (toename),Positief getest (toename),Ziekenhuisopnames (toename),Overleden (toename),Personen,Key
Key,,,,,,,,,,,,,,,,,,,,,
2020-02-27,2020-02-27,NL,1,13,13,0,2,64,61,0,...,1,0,0,0,0,0,0,0,17597607,2020-02-27
2020-02-28,2020-02-28,NL,0,14,14,0,4,68,65,0,...,9,0,0,1,8,0,0,0,17597607,2020-02-28
2020-02-29,2020-02-29,NL,1,15,15,0,1,69,65,0,...,10,0,0,1,1,0,0,0,17597607,2020-02-29
2020-03-01,2020-03-01,NL,2,17,17,0,7,76,72,0,...,20,0,0,2,10,0,0,0,17597607,2020-03-01
2020-03-02,2020-03-02,NL,0,17,16,0,10,87,83,0,...,34,0,0,-1,14,0,0,0,17597607,2020-03-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-18,2022-03-18,NL,15,17774,161,4900,225,90267,2151,11379,...,1823,17,256,-2,22,54216,0,19,17597607,2022-03-18
2022-03-19,2022-03-19,NL,14,17789,163,4907,174,90437,2024,11397,...,1665,10,219,-13,-158,41456,0,11,17597607,2022-03-19
2022-03-20,2022-03-20,NL,16,17805,162,4912,151,90586,2039,11404,...,1723,18,179,-2,58,34823,0,8,17597607,2022-03-20


In [10]:
if knack:
  await knack.publish(dagoverzicht, 'Dagoverzicht', Cache)

infinities:


Empty DataFrame
Columns: []
Index: []


nan:


Empty DataFrame
Columns: []
Index: []


updating knack


...

updating: [{'Key': 1, 'Updating Dagoverzicht': 'false'}]


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.10s/it]

100%|██████████| 1/1 [00:02<00:00,  2.10s/it]


rate limit: 7 
LaatsteUpdate API calls: create: 0, read: 0, update: 1, delete: 0, backoff: 0
